In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
FOLDERNAME = "Colab\ Notebooks/fetch-data"
%cd drive/MyDrive/$FOLDERNAME

Mounted at /content/drive


In [ ]:
# !pip install Unidecode

In [ ]:
import pandas as pd
import numpy as np
import re
import json, ast
from unidecode import unidecode
from sklearn.preprocessing import OneHotEncoder

In [ ]:
FILENAME = 'merged_apple_swe.csv'
SPLIT_FILE_NAME = FILENAME.split('_')
COMPANY = SPLIT_FILE_NAME[1].capitalize()
ROLE = SPLIT_FILE_NAME[2].split('.')[0]
PATH = f'raw/{COMPANY}/'
OUTPUT = f'cleaned/LGBM/cleaned_{COMPANY.lower()}_{ROLE}.csv'

In [ ]:
LABELS = {
    'da': 0,
    'ds': 1,
    'pm': 2,
    'swe': 3
}

# Raw Dataframe

In [ ]:
def parse_json_list(x):
    if pd.isna(x) or not isinstance(x, str) or x.strip() in ['', '[]']:
        return []
    s = x.strip()
    try:
        return json.loads(s)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(s)
        except (ValueError, SyntaxError):
            return []

df = pd.read_csv(
    (PATH+FILENAME),
    converters={
        'education'     : parse_json_list,
        'experience'    : parse_json_list,
        'certifications': parse_json_list,
        'projects'      : parse_json_list,
        'publications'  : parse_json_list,
        'courses'       : parse_json_list
    }
)
df = df.drop_duplicates(subset='id')

# Education (edu)

#### Degree

In [ ]:
def classify_degree(deg):
    """
    Return (degree_cat, new_cert, new_course)：
      - degree_cat: "Bachelor's"/"Master's"/"PhD"/"Associate"/"Other"/"Extracurricular" or None
      - new_cert: Return deg or None based on keywords matching
      - new_course: Return deg or None based on keywords matching
    """
    if not isinstance(deg, str) or not deg.strip():
        return (None, None, None)
    s = deg.strip()

    # Official Degree
    deg_patterns = {
        "Bachelor's": re.compile(
            r"\bbachelor|学士|學士|licenciatura|laurea|licence"
            r"|(?<!m)BA\b|(?<!m)B\.A?\b|(?<!m)BS\b|(?<!m)B\.S\.?\b|B\.F\.A\.?|BFA\b|BTech\b|B\.Tech?"
            r"|BE\b|BEng\b|B\.Eng\b|BSc\b|B\.Sc\b|B\.Com?|A\.B?|",
            re.IGNORECASE
        ),
        "Master's":   re.compile(
            r"\bmaster|硕士|碩士|máster|magistrale"
            r"|MS\b|M\.S\.?\b|MA\b|M\.A\.?\b"
            r"|MBA\b|M\.B\.A\.?\b|MBS\b|M\.B\.S\.?|M\.Tech?|MTech"
            r"|MSc\b|M\.Sc\b|MPhil|M\.Phil\b|M\.F\.A\.?|MFA\b",
            re.IGNORECASE
        ),
        "PhD":        re.compile(
            r"\bphd\b|博士|doctorad|dottorato|doctorat|doktor|doctor"
            r"|PhD|Ph\.?D\.?\b|DFA\b|D\.F\.A\b|J\.D\.?\b|Dphil\b|D\.Phil\.?|MD|DDS|M\.D\.?|D\.D\.S\b",
            re.IGNORECASE
        ),
        "Associate":  re.compile(
            r"\bassociate|副学士|副學士|專科|二技|五專|t[eé]cnico superior"
            r"|AS\b|A\.S\.?\b|AA\b|A\.A\.?\b|BTS\b|DUT\b|Fachhochschul",
            re.IGNORECASE
        ),
    }

    for cat, pat in deg_patterns.items():
        if pat.search(s):
            return (cat, None, None)

    # Certificates
    cert_pattern = re.compile(r"certificat|diploma|nanodegree", re.IGNORECASE)
    if cert_pattern.search(s):
        return (None, s, None)

    # Short-term studies
    short_course_re = re.compile(
        r"bootcamp|workshop|program(me)?|summer|exchange|semester|abroad", re.IGNORECASE
    )
    if short_course_re.search(s):
        return ("Extracurricular", None, None)

    # Courses
    long_course_kw = re.compile(
        r"Studies|Science|Engineering|Economics|Mathematics|Media|Computational|Physics|"
        r"Cyber|Management|Electronic|Préparatoire",
        re.IGNORECASE
    )
    if long_course_kw.search(s):
        return (None, None, s)

    # Other
    return ("Other", None, None)

In [ ]:
# Generate "edu"
edu = (
    df[['id','education']]
      .explode('education')
      .dropna(subset=['education'])
      .reset_index(drop=True)
)
edu = pd.DataFrame({
    'id':         edu['id'],
    'school':     edu['education'].map(lambda x: x.get('title') if isinstance(x, dict) else None),
    'degree_raw': edu['education'].map(lambda x: x.get('degree') if isinstance(x, dict) else None),
    'field':      edu['education'].map(lambda x: x.get('field') if isinstance(x, dict) else None),
})

# apply classify_degree, expand=True
edu[['degree_cat','new_cert','new_course']] = (
    edu['degree_raw']
       .apply(classify_degree)
       .apply(pd.Series)
)

degree_code = {
    "PhD": 0, "Master's": 1, "Bachelor's": 2,
    "Associate": 3, "Other": 5, "Extracurricular": 4
}
edu['degree_code'] = edu['degree_cat'].map(degree_code).fillna(6).astype(int)

cert_orig = (
    df[['id','certifications']].explode('certifications')
      .dropna(subset=['certifications'])
      .pipe(lambda d: pd.DataFrame({
          'id': d['id'],
          'cert_title': d['certifications'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
      }))
)
course_orig = (
    df[['id','courses']].explode('courses')
      .dropna(subset=['courses'])
      .pipe(lambda d: pd.DataFrame({
          'id': d['id'],
          'course_title': d['courses'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
      }))
)

# Get certifications / courses from edu
cert_new   = edu[['id','new_cert']].dropna(subset=['new_cert']).rename(columns={'new_cert':'cert_title'})
course_new = edu[['id','new_course']].dropna(subset=['new_course']).rename(columns={'new_course':'course_title'})

# Concatinate & drop duplicates
cert_all   = pd.concat([cert_orig,   cert_new],   ignore_index=True).drop_duplicates()
course_all = pd.concat([course_orig, course_new], ignore_index=True).drop_duplicates()

# Aggregate the data & merge back to df
cert_map   = cert_all.groupby('id')['cert_title'].agg(list)
course_map = course_all.groupby('id')['course_title'].agg(list)

df['certifications'] = df['id'].map(lambda i: cert_map.get(i, []))
df['courses']        = df['id'].map(lambda i: course_map.get(i, []))

#### Field

In [ ]:
# field → numbers
def map_field(f: str) -> int:
    if not isinstance(f, str) or not f.strip():
        return 5
    s = f.strip().lower()
    # (1) CS-Related
    if "computer" in s or "software engineering" in s or "information engineering" in s or "artificial intelligence" in s:
        return 0
    # (2) EE/ECE
    ee_patterns = [
        "electrical and electronics engineering",
        "electrical engineering",
        "electronics and communications engineering",
        "electronics engineering",
        r"\belectrical\b",
        r"\bee\b",
        r"\bece\b"
    ]
    if any(re.search(pat, s) for pat in ee_patterns):
        return 1
    # (3) Information / Data / Stats / Cyber / Math / Physics
    kw3 = ["information","informatics","data","statistic","cyber","mathematics","physics"]
    if any(kw in s for kw in kw3):
        return 2
    # (4) Business / Economics / Finance / Management / Marketing / Accounting
    kw4 = ["business","economics","finance","management","marketing","accounting"]
    if any(kw in s for kw in kw4):
        return 3
    # (5) Other STEM
    if "engineering" in s or "science" in s:
        return 4
    # (6) Other
    return 5

edu['field_code'] = edu['field'].apply(map_field)

#### School Tier

In [ ]:
# Load THE CS Top100 List
rank_df = pd.read_csv('the_cs_top100.csv')  # 包含 rank, university, country

# Normalize school names
def normalize(name: str) -> str:
    if not isinstance(name, str):
        return ""
    # Transfer non-Latin alphabets languages into latin alphabets
    s = unidecode(name)
    # Keep words only before '-'
    s = re.split(r'\s*[-–—]\s*', s, maxsplit=1)[0]
    # Remove common university-related words
    s = s.lower()
    s = re.sub(r'\b(university|institute|college|da xue|school|of|and)\b', '', s)
    # Keep spaces & alphabets only
    s = re.sub(r'[^a-z0-9 ]+', ' ', s)
    # Concatenate spaces
    s = re.sub(r'\s+', ' ', s).strip()
    return s

# Normalize school names in both school list & edu
rank_df['school_norm'] = rank_df['university'].map(normalize)
edu['school_norm']     = edu['school'].map(normalize)

# Merge rankings to edu
edu = edu.merge(
    rank_df[['school_norm','rank']],
    on='school_norm',
    how='left'
)

# Define Tier
def rank_to_tier(r):
    if pd.isna(r):        return 'Other'
    r = int(r)
    if r <= 20:           return 'Top20'
    if r <= 50:           return 'Top50'
    if r <= 100:          return 'Top100'
    return 'Other'

edu['school_tier'] = edu['rank'].apply(rank_to_tier)

tier_map = {
    'Top20':   0,
    'Top50':   1,
    'Top100':  2,
    'Other':   3
}

# Create tier_code
edu['tier_code'] = edu['school_tier'].map(tier_map).fillna(3).astype(int)

In [ ]:
edu = edu[[
    'id','degree_code','field_code','tier_code'
]]

# Experience (exp)

In [ ]:
import pandas as pd
import re

# Assume COMPANY is defined (one of 'amazon','apple','google','meta','microsoft')
COMPANY = COMPANY.lower()

# Normalize current company names to lowercase and strip whitespace
df['current_company_name'] = (
    df['current_company_name']
      .fillna('')
      .str.lower()
      .str.strip()
)

# Define synonyms for each FAAMG company
synonyms = {
    'amazon':  ['amazon', 'aws'],
    'apple':   ['apple'],
    'google':  ['google', 'alphabet'],
    'meta':    ['meta', 'facebook'],
    'microsoft': ['microsoft']
}

# Filter master DataFrame to only rows whose current company matches COMPANY synonyms
keys = synonyms.get(COMPANY, [COMPANY])
df = df[df['current_company_name']
        .apply(lambda name: any(k in name for k in keys))
     ].reset_index(drop=True)

# Explode the 'experience' list into one row per entry
exp = (
    df[['id', 'current_company_name', 'experience']]
     .explode('experience')
     .reset_index(drop=True)
)

# Map any company name containing FAANG keywords into a canonical FAANG label
def map_faang(name: str) -> str:
    s = name.lower()
    if 'amazon' in s or 'aws' in s:
        return 'amazon'
    if 'meta' in s or 'facebook' in s:
        return 'meta'
    if 'google' in s or 'alphabet' in s:
        return 'google'
    if 'microsoft' in s or 'msft' in s:
        return 'microsoft'
    if 'apple' in s:
        return 'apple'
    return None

exp['faang_comp'] = exp['current_company_name'].apply(map_faang)

# Keep only rows that were mapped to a FAANG company
exp = exp[exp['faang_comp'].notna()].copy()

# Use that FAANG label as our normalized company column
exp['comp_norm'] = exp['faang_comp']

# Extract subfields from the experience dict
exp['exp_company']     = exp['experience'].apply(lambda x: x.get('company')     if isinstance(x, dict) else None)
exp['exp_title']       = exp['experience'].apply(lambda x: x.get('title')       if isinstance(x, dict) else None)
exp['exp_description'] = exp['experience'].apply(lambda x: x.get('description') if isinstance(x, dict) else None)
exp['exp_duration']    = exp['experience'].apply(lambda x: x.get('duration')    if isinstance(x, dict) else None)
exp['exp_positions']   = exp['experience'].apply(
    lambda x: [p.get('subtitle') for p in x.get('positions', [])] if isinstance(x, dict) else []
)

# Flag which rows are the “current company” vs. past experiences
exp['is_current_company'] = exp['comp_norm'] == exp['current_company_name']
exp['order']              = exp.groupby('id').cumcount()
first_current = exp[exp['is_current_company']].groupby('id')['order'].min()
exp['is_past_exp'] = exp.apply(
    lambda r: r['order'] < first_current.get(r['id'], float('inf')),
    axis=1
)

# Mark past experiences at Fortune 500 companies
fortune500 = pd.read_csv('fortune500.csv')
f500_set   = set(fortune500['Company'].str.lower().str.strip())
exp['is_f500'] = exp.apply(
    lambda r: 1 if (r['is_past_exp'] and r['comp_norm'] in f500_set) else 0,
    axis=1
)

# Parse “X years Y months” strings into total years (float)
def parse_duration(s: str) -> float:
    if not isinstance(s, str):
        return 0.0
    y_match = re.search(r'(\d+)\s*year', s)
    m_match = re.search(r'(\d+)\s*month', s)
    years   = int(y_match.group(1)) if y_match else 0
    months  = int(m_match.group(1)) if m_match else 0
    return years + months / 12

exp['duration_years'] = exp['exp_duration'].apply(parse_duration)

# Bucketize duration into categorical codes
def bucket_duration(years: float) -> int:
    if years > 10:
        return 0
    if years > 5:
        return 1
    if years > 2:
        return 2
    return 3

exp['duration_bucket'] = exp['duration_years'].apply(bucket_duration)

# Select the final columns you want to keep
exp = exp[[
    'id', 'exp_company', 'exp_title', 'exp_description',
    'exp_duration', 'duration_years', 'duration_bucket', 'exp_positions',
    'is_current_company', 'is_past_exp', 'is_f500', 'comp_norm'
]]

# One-hot encode the normalized FAANG company labels—and force all five columns
ALL_COMP = ['amazon','apple','google','meta','microsoft']
ALL_COLS = [f'comp_{c}' for c in ALL_COMP]

# generate dummies from comp_norm
comp_ohe = pd.get_dummies(exp['comp_norm'], prefix='comp')

# reindex to ensure all five comp_* columns exist (missing ones filled with 0)
comp_ohe = comp_ohe.reindex(columns=ALL_COLS, fill_value=0)

# concat back and cast to int
exp = pd.concat([exp, comp_ohe], axis=1)
exp[ALL_COLS] = exp[ALL_COLS].astype(int)

# Certifications (cert)

In [ ]:
cert = (
    df[['id','certifications']]
      .explode('certifications')
      .dropna(subset=['certifications'])
)
cert = pd.DataFrame({
    'id': cert['id'],
    'cert_title': cert['certifications'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
})

# Projects (proj)

In [ ]:
proj = (
    df[['id','projects']]
      .explode('projects')
      .dropna(subset=['projects'])
)
proj = pd.DataFrame({
    'id': proj['id'],
    'proj_title':       proj['projects'].map(lambda x: x.get('title')),
    'proj_description': proj['projects'].map(lambda x: x.get('description')),
})

# Publications (pub)

In [ ]:
pub = (
    df[['id','publications']]
      .explode('publications')
      .dropna(subset=['publications'])
)
pub = pd.DataFrame({
    'id':      pub['id'],
    'pub_title': pub['publications'].map(lambda x: x.get('title')),
})

# Courses (course)

In [ ]:
course = (
    df[['id','courses']]
      .explode('courses')
      .dropna(subset=['courses'])
)
course = pd.DataFrame({
    'id':         course['id'],
    'course_title': course['courses'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
})

# Merge tables

In [ ]:
id_df = pd.DataFrame({'id': df['id'].unique()})

#### edu

In [ ]:
# Get best education info
best_idx = edu.groupby('id')['degree_code'].idxmin()
best_edu = (
    edu.loc[best_idx, ['id','degree_code','tier_code']]
       .drop_duplicates(subset=['id'])
)

# Calculate field mode
field_mode = (
    edu.groupby('id')['field_code']
       .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
       .reset_index()
       .rename(columns={'field_code':'field_mode'})
)

# Left join Merge best_edu & field_mode to id_df
edu_agg = (
    id_df
      .merge(best_edu,    on='id', how='left')
      .merge(field_mode,  on='id', how='left')
)

#### exp

In [ ]:
comp_cols = [f'comp_{c}' for c in ALL_COMP]

exp_agg = exp.groupby('id').agg(
    total_past_exp   = ('is_past_exp',    'sum'),
    f500_past_exp    = ('is_f500',        'sum'),
    sum_years        = ('duration_years', 'sum'),
    mean_years       = ('duration_years', 'mean'),
    **{col: (col, 'max') for col in comp_cols}
).reset_index()

exp_agg[comp_cols] = exp_agg[comp_cols].astype(int)
print(exp_agg.columns.tolist())

['id', 'total_past_exp', 'f500_past_exp', 'sum_years', 'mean_years', 'comp_amazon', 'comp_apple', 'comp_google', 'comp_meta', 'comp_microsoft']


#### proj

In [ ]:
# Create proj_text: combine title & description
proj['proj_text'] = (
    proj['proj_title'].fillna('') + ' ' +
    proj['proj_description'].fillna('')
).str.strip()
proj_combined = (
    proj.groupby('id')['proj_text']
        .apply(lambda texts: ' '.join([t for t in texts if t]))
        .reset_index()
)
proj_agg = (id_df.merge(proj_combined, on='id', how='left'))

#### course

In [ ]:
course = (
    df[['id','courses']]
      .explode('courses')
      .dropna(subset=['courses'])
)
course = pd.DataFrame({
    'id': course['id'],
    'course_title': course['courses'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
})
course_agg = (
    course.groupby('id')['course_title']
          .agg(lambda titles: ' [SEP] '.join(titles))
          .reset_index()
          .rename(columns={'course_title': 'course_text'})
)
course_count = course.groupby('id').size().reset_index(name='n_course')

#### cert

In [ ]:
cert = (
    df[['id','certifications']]
      .explode('certifications')
      .dropna(subset=['certifications'])
)
cert = pd.DataFrame({
    'id': cert['id'],
    'cert_title': cert['certifications'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
})
cert_agg = (
    cert.groupby('id')['cert_title']
        .agg(lambda titles: ' [SEP] '.join(titles))
        .reset_index()
        .rename(columns={'cert_title': 'cert_text'})
)
cert_count = cert.groupby('id').size().reset_index(name='n_cert')

#### pub

In [ ]:
pub = (
    df[['id','publications']]
      .explode('publications')
      .dropna(subset=['publications'])
)
pub = pd.DataFrame({
    'id': pub['id'],
    'pub_title': pub['publications'].map(lambda x: x.get('title') if isinstance(x, dict) else x)
})
pub_agg = (
    pub.groupby('id')['pub_title']
       .agg(lambda titles: ' [SEP] '.join(titles))
       .reset_index()
       .rename(columns={'pub_title': 'pub_text'})
)
pub_count = pub.groupby('id').size().reset_index(name='n_pub')

In [ ]:
# Merge to df_final
df_final = (
    df
    .merge(cert_agg,   on='id', how='left')
    .merge(cert_count, on='id', how='left')
    .merge(course_agg, on='id', how='left')
    .merge(course_count,on='id', how='left')
    .merge(pub_agg,    on='id', how='left')
    .merge(pub_count,  on='id', how='left')
    .merge(edu_agg,    on='id', how='left')
    .merge(exp_agg,    on='id', how='left')
    .merge(proj_agg,   on='id', how='left')
)

# Fiil 0 to empty spaces
df_final[['n_cert','n_course','n_pub']] = (
    df_final[['n_cert','n_course','n_pub']]
    .fillna(0)
    .astype(int)
)

columns_to_drop = [
    'education', 'experience', 'projects', 'courses',
    'certifications', 'publications', 'name',
    'country_code', 'position', 'url',
    'current_company_company_id', 'current_company_name'
]

df_final = df_final.drop(columns=columns_to_drop, errors='ignore')
df_final['label'] = LABELS[ROLE]

In [ ]:
df_final.to_csv(OUTPUT, index=False)
print(OUTPUT + ' is saved.')

cleaned/LGBM/cleaned_apple_swe.csv is saved.
